In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import tensorflow as tf
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D,AlphaDropout,SeparableConv2D, SpatialDropout2D, MaxPooling2D
from tensorflow.keras import regularizers
from keras.utils import np_utils
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping
%matplotlib inline

In [ ]:
#Loading Labels
traindf = pd.read_excel('/content/drive/My Drive/Paper/UTKFace/traindf.xlsx')
valdf = pd.read_excel('/content/drive/My Drive/Paper/UTKFace/valdf.xlsx')
testdf = pd.read_excel('/content/drive/My Drive/Paper/UTKFace/testdf.xlsx')
traindf.head()

In [ ]:
import seaborn as sns

plt.figure(figsize=(13,6))
sns.kdeplot(traindf['age'], label='train', gridsize=100)
sns.kdeplot(valdf['age'], label='val',  gridsize=100)
sns.kdeplot(testdf['age'], label='test', gridsize=100)
plt.show();

In [ ]:
#Loading Features Extracted using ResNet50
fx_train_vgg= np.load('/content/drive/My Drive/Paper/UTKFace/fx_train_vgg.npy')
fx_val_vgg = np.load('/content/drive/My Drive/Paper/UTKFace/fx_val_vgg.npy')
fx_test_vgg = np.load('/content/drive/My Drive/Paper/UTKFace/fx_test_vgg.npy')
print("Loaded!")

Loaded!


In [ ]:
x_train= np.load('/content/drive/My Drive/Paper/UTKFace/x_train.npy')
x_val = np.load('/content/drive/My Drive/Paper/UTKFace/x_val.npy')
x_test = np.load('/content/drive/My Drive/Paper/UTKFace/x_test.npy')

Age Estimation Model

In [ ]:
print('Train : '+str(fx_train_vgg.shape))
print('Val : '+str(fx_val_vgg.shape))
print('Test : '+str(fx_test_vgg.shape))

In [ ]:
model = Sequential()

model.add(BatchNormalization())
model.add(SpatialDropout2D(0.2))
model.add(SeparableConv2D(512,(3,3),padding='same',activation='relu',depthwise_initializer='he_uniform',pointwise_initializer='he_uniform'))

model.add(BatchNormalization())
model.add(SpatialDropout2D(0.4))
model.add(SeparableConv2D(512,(3,3),padding='same',activation='relu',depthwise_initializer='he_uniform',pointwise_initializer='he_uniform'))
model.add(MaxPooling2D(2,2))


model.add(Flatten())

model.add(BatchNormalization())
model.add(Dropout(0.8))
model.add(Dense(1024,activation='elu',kernel_initializer='he_uniform'))

model.add(BatchNormalization())
model.add(Dropout(0.8))
model.add(Dense(512,activation='elu',kernel_initializer='he_uniform'))

model.add(BatchNormalization())
#model.add(Dropout(0.5))
model.add(Dense(128,activation='elu',kernel_initializer='he_uniform'))

model.add(BatchNormalization())
model.add(Dense(1,activation='relu',kernel_initializer='he_uniform'))

In [ ]:
sgd = tf.keras.optimizers.SGD(
    learning_rate=1e-2, momentum=0.9, nesterov=True, name="SGD")

adam = tf.keras.optimizers.Adam(
    learning_rate=4e-4,
    amsgrad=True,)

adamax = tf.keras.optimizers.Adamax(learning_rate=1e-3)

ftrl = tf.keras.optimizers.Ftrl(
    learning_rate=0.01,
    learning_rate_power=-0.5,
    initial_accumulator_value=0.1,
    l1_regularization_strength=0.0,
    l2_regularization_strength=0.0,
    name="Ftrl",
    l2_shrinkage_regularization_strength=0.0,
)

adadelta = tf.keras.optimizers.Adadelta(
    learning_rate=1e-3, rho=0.95, epsilon=1e-08, name="Adadelta"
)

adagrad=tf.keras.optimizers.Adagrad(
    learning_rate=0.001,
    initial_accumulator_value=0,
    epsilon=1e-07,
    name="Adagrad",
)

model.compile(optimizer=adam,loss='mse',metrics=['mae'])

In [ ]:
model.fit(fx_train_vgg,traindf['age'].values ,epochs=5, validation_data=(fx_val_vgg,valdf['age'].values),batch_size=128)

In [ ]:
model.evaluate(fx_test_vgg,testdf['age'].values)

In [ ]:
model.save('/content/drive/My Drive/Paper/UTKFace/Age Estimation/VGG16/model.h5')

In [ ]:
model.predict(fx_test_vgg).shape

Ensemble

In [ ]:
from keras.models import load_model

G_res = load_model('/content/drive/My Drive/Paper/UTKFace/Gender/ResNet50/9464.h5')
G_res.compile("Adam","binary_crossentropy",['acc'])

G_sen = load_model('/content/drive/My Drive/Paper/UTKFace/Gender/SeNet50/9494.h5')
G_sen.compile("Adam","binary_crossentropy",['acc'])

G_vgg = load_model('/content/drive/My Drive/Paper/UTKFace/Gender/VGG16/9367.h5')
G_vgg.compile("Adam","binary_crossentropy",['acc'])

In [ ]:
fx_test_res_e = np.load('/content/drive/My Drive/Paper/UTKFace/fx_test_res.npy')
fx_test_sen_e = np.load('/content/drive/My Drive/Paper/UTKFace/fx_test_sen.npy')
fx_test_vgg_e = np.load('/content/drive/My Drive/Paper/UTKFace/fx_test_vgg.npy')

In [ ]:
pred_res = G_res.predict(fx_test_res_e)
pred_sen = G_sen.predict(fx_test_sen_e)
pred_vgg = G_vgg.predict(fx_test_vgg_e)

In [ ]:
pred_res.shape, pred_sen.shape, pred_vgg.shape

((2371, 1), (2371, 1), (2371, 1))

In [ ]:
y_pred = (pred_res+pred_sen+pred_vgg)/3

In [ ]:
from sklearn.metrics import classification_report
report = classification_report(testdf['gender'].values,y_pred.round())
print(report)

In [ ]:
from sklearn.metrics import roc_auc_score

auc_vgg = roc_auc_score(testdf['gender'].values,pred_vgg.round())
auc_res = roc_auc_score(testdf['gender'].values,pred_res.round())
auc_sen = roc_auc_score(testdf['gender'].values,pred_sen.round())
auc_en = roc_auc_score(testdf['gender'].values,y_pred.round())
print('ROC_AUC_SCORE : \n')
print('ResNet50: ',auc_res)
print('SeNet50: ',auc_sen)
print('VGG16: ',auc_vgg)
print('Ensemble: ',auc_en)

ROC_AUC_SCORE : 

ResNet50:  0.9465709899376933
SeNet50:  0.9491653231726701
VGG16:  0.9361109804441441
Ensemble:  0.9477423599024417


In [ ]:
from sklearn.metrics import accuracy_score 

acc_vgg = accuracy_score(testdf['gender'].values,pred_vgg.round())
acc_res = accuracy_score(testdf['gender'].values,pred_res.round())
acc_sen = accuracy_score(testdf['gender'].values,pred_sen.round())
acc_en = accuracy_score(testdf['gender'].values,y_pred.round())
print('ACCURACY_SCORE : \n')
print('ResNet50: ',acc_res)
print('SeNet50: ',acc_sen)
print('VGG16: ',acc_vgg)
print('Ensemble: ',acc_en)

ACCURACY_SCORE : 

ResNet50:  0.9464361029101644
SeNet50:  0.9493884436946436
VGG16:  0.9367355546183045
Ensemble:  0.9481231547870097
